In [1]:
import os
import sys
import urllib.request
# from dotenv import load_dotenv
# load_dotenv()
import pandas as pd
import requests

In [2]:
!pip install openpyxl

In [3]:
import warnings

# 경고 메시지 무시 설정
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [22]:
import pandas as pd
import json
import glob
import os

# 엑셀 파일들이 들어 있는 최상위 폴더 경로 (리눅스 경로로 수정)
base_dir = r"/mnt/c/study/KISTI_AI/023.국회 회의록 기반 지식검색 데이터/3.개방데이터/1.데이터/Training/01.원천데이터"

# 모든 하위 폴더 내 엑셀 파일 경로 검색
excel_files = glob.glob(os.path.join(base_dir, '**', '*.xlsx'), recursive=True)

qa_data = []  # JSON으로 저장할 데이터 리스트

# 모든 엑셀 파일 읽기
for file_path in excel_files:
    try:
        # 엑셀 파일 읽기
        df = pd.read_excel(file_path)

        question_info = None  # 현재 질문 정보를 저장할 변수

        # 각 행을 순회하며 질문(Q)과 답변(A) 추출
        for idx, row in df.iterrows():
            if row['질의응답'] == 'Q':  # 질문일 때
                question_info = {
                    "question": row['발언내용'],
                    "회의번호": row['회의번호'],
                    "질의응답번호": row['질의응답번호'],
                    "회의구분": row['회의구분'],
                    "위원회": row['위원회'],
                    "회의일자": row['회의일자'],
                    "질문자": row['의원ID'],
                    "질문자_ISNI": row['ISNI']
                }
            elif row['질의응답'] == 'A' and question_info is not None:  # 답변일 때
                answer_info = {
                    "answer": row['발언내용'],
                    "답변자": row['의원ID'],
                    "답변자_ISNI": row['ISNI']
                }
                # 질문과 답변을 연결하여 하나의 JSON 객체로 만듦
                qa_data.append({**question_info, **answer_info})
                question_info = None  # 사용 후 질문 정보를 초기화

    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
    except pd.errors.EmptyDataError:
        print(f"빈 파일이므로 건너뜁니다: {file_path}")
    except Exception as e:
        print(f"파일 처리 중 오류 발생 ({file_path}): {e}")

# 라벨링 데이터 파일 저장 경로 (리눅스 경로로 수정)
save_dir = r"/mnt/c/study/KISTI_AI/023.국회 회의록 기반 지식검색 데이터/3.개방데이터/1.데이터/Training/02.라벨링데이터"
save_path = os.path.join(save_dir, '라벨링데이터.json')

# 디렉토리가 존재하지 않으면 생성
os.makedirs(save_dir, exist_ok=True)

# JSON 파일로 저장
try:
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(qa_data, f, ensure_ascii=False, indent=4)
    print("모든 파일 처리가 완료되었습니다.")
except Exception as e:
    print(f"JSON 파일 저장 중 오류 발생: {e}")


모든 파일 처리가 완료되었습니다.


In [23]:
import json
import os

# 라벨링된 JSON 파일 경로 (리눅스/WSL 경로로 수정)
file_path = "/mnt/c/study/KISTI_AI/023.국회 회의록 기반 지식검색 데이터/3.개방데이터/1.데이터/Training/02.라벨링데이터/라벨링데이터.json"

# JSON 파일 로드 및 예외 처리
try:
    # 파일 존재 여부 확인
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다. 경로를 확인해주세요: {file_path}")

    # JSON 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

        # 1. JSON 파일에서 일부 샘플을 확인 (처음 5개 데이터 출력)
        print("데이터 샘플 확인 (처음 5개 질문-답변 쌍):")
        for i, item in enumerate(data[:5]):
            print(f"샘플 {i + 1}:")
            print(json.dumps(item, indent=4, ensure_ascii=False))
            print("-" * 50)

        # 2. 총 질문-답변 쌍의 개수 확인
        print(f"\n총 질문-답변 쌍의 개수: {len(data)}")

        # 3. 필드 유효성 검사
        required_fields = {"question", "answer", "회의번호", "질의응답번호", "회의구분", "위원회", "회의일자", "질문자", "질문자_ISNI", "답변자", "답변자_ISNI"}
        print("\n필드 유효성 검사:")
        for i, item in enumerate(data):
            missing_fields = required_fields - item.keys()  # 필요한 필드 중 누락된 필드 찾기
            if missing_fields:
                print(f"항목 {i}에 필요한 필드가 누락되었습니다: {missing_fields}")
        
        # 4. 질문-답변 쌍의 일관성 확인
        print("\n질문-답변 쌍 일관성 확인:")
        for i, item in enumerate(data):
            if not item.get("question") or not item.get("answer"):
                print(f"항목 {i}에 질문 또는 답변이 없습니다.")

except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다. 경로를 확인해주세요: {file_path}")
except json.JSONDecodeError:
    print("JSON 파일을 읽는 중 오류가 발생했습니다. 파일 형식이 올바른지 확인해주세요.")
except Exception as e:
    print(f"오류 발생: {e}")


데이터 샘플 확인 (처음 5개 질문-답변 쌍):
샘플 1:
{
    "question": "우리 사장이나 간부들이 이것을 대단히 소홀하게 중요하지 않게 생각하는데 끝까지 내 이야기에 대한 답변이 나오지 않았습니다.  공역문제인데 아까 공역을 전부 예를 들었습니다마는 가령 미공군 매향리 사격장, 한ㆍ미합동훈련장 또 여주 공군사격장, 수도권 비행금지구역, 휴전선 비행금지구역 이렇게 공역이 되어 있는데 앞으로 허브공항으로서 많은 비행기가 오는 곳에 공역이 도처에 있어요. 과연 그 공역을 피해 갈 수 있을 것인가에 대한 어려움을 우리 건교부하고 미8군하고 국방부하고 협의해서 공역문제는 사전에 합의를 보아야 할 문제인데 이것에 대해서 사장은 어떤 생각을 가지고 있느냐고 물어 보았습니다. 답변을 해주십시오.",
    "회의번호": 30045,
    "질의응답번호": 1,
    "회의구분": "국정감사",
    "위원회": "건설교통위원회",
    "회의일자": "2000年10月19日(木)",
    "질문자": "3337",
    "질문자_ISNI": "0000000463657210",
    "answer": "서면으로 답변을 드리려고 했습니다마는 공역문제는 너무 중요한 문제입니다. 그리고 당장은 아니지만 2, 3년 앞을 내다보면 매우 심각한 문제입니다. 미국 정부에서도 그 심각성을 인정하고 있습니다. 그래서 제가 알기로는 건교부에서 국방부, 미 당국과도 협의를 이미 진행하고 있습니다.   그래서 공역을 변경하기로 일부 합의되어서 저희 공항개항과 더불어 현재의 공역보다도 많이 여유를 갖게 되어 있습니다. 그리고 근본적인 문제는 앞으로 한ㆍ미간 또는 국내에서 국방부와 협의가 계속 진행될 것으로 알고 있습니다.",
    "답변자": " ",
    "답변자_ISNI": " "
}
--------------------------------------------------
샘플 2:
{
    "question": "제가 아까 한 질의는 아주 근본적인 질의거든요

In [ ]:
다음 단계: 모델 학습 준비 및 학습
데이터 전처리 및 준비:

생성한 라벨링된 JSON 파일을 이용해 모델 학습에 적합한 형태로 변환해야 합니다.
데이터를 질문-답변 쌍 형식으로 학습 데이터셋에 사용합니다.
데이터를 나누어 **훈련 데이터(Training Set)**와 **검증 데이터(Validation Set)**로 나누는 작업을 합니다.
모델 선택:

질의응답 모델로 학습할 수 있는 사전 학습된 모델을 선택합니다. 대표적인 예시로는 다음과 같은 모델들이 있습니다:
BERT (Bidirectional Encoder Representations from Transformers): 일반적으로 질의응답 태스크에 자주 사용됩니다.
GPT (Generative Pre-trained Transformer): 대화형 모델과 자유 형식의 질문에 대해 답변을 생성하는 데 유용합니다.
Hugging Face의 Transformers 라이브러리를 사용하면 다양한 사전 학습된 모델을 쉽게 사용할 수 있습니다.
환경 설정 및 모델 학습:

Python에서 모델을 학습할 수 있는 환경을 설정합니다.


In [38]:
# !pip install nltk

In [34]:
# !pip install transformers
# !pip install torch
# !pip install torch
# !pip install torch transformers konlpy

In [35]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [36]:
import torch
print(torch.__version__)


2.4.1+cu121


In [32]:
import json
import re
import torch
from transformers import BertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from konlpy.tag import Mecab

# Mecab 형태소 분석기 로드 (한국어 데이터의 경우)
mecab = Mecab()

# JSON 파일 경로 설정 (리눅스/WSL 경로로 수정)
file_path = "/mnt/c/study/KISTI_AI/023.국회 회의록 기반 지식검색 데이터/3.개방데이터/1.데이터/Training/02.라벨링데이터/라벨링데이터.json"

# JSON 파일 로드
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# BERT Tokenizer 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 불용어 정의
stop_words = [
    "것", "있다", "하다", "입니다", "그리고", "하지만", "또한", "그런데", "저는", "우리는", "그래서", "이것", "저것", "그것",
    "다시", "모든", "각각", "모두", "어느", "몇몇", "이런", "저런", "그런", "어떤", "특히", "즉", "또", "이후", "때문에", "통해서",
    "같은", "많은", "따라서", "등", "경우", "관련", "대해", "의해", "이기", "대한", "그리고", "라고", "이라는", "에서", "부터", "까지",
    "와", "과", "으로", "에", "의", "를", "가", "도", "로", "에게", "만", "뿐", "듯", "제", "내", "저", "그", "할", "수", "있", "같",
    "되", "보다", "아니", "아닌", "이", "있어서", "입니다", "있습니다", "합니다", "입니까", "같습니다", "아닙니다", "라는", "그러므로", 
    "입니다만", "때문입니다", "라고요", "그러하다", "하고", "이와"
]

# 데이터 정제 및 전처리 함수 정의
def preprocess_text(text):
    # 1. 특수 문자 및 불필요한 공백 제거
    text = re.sub(r"[^가-힣a-zA-Z0-9\s]", "", text)  # 한글, 영문, 숫자, 공백만 남기기
    text = re.sub(r"\s+", " ", text).strip()  # 중복 공백 제거 및 양끝 공백 제거

    # 2. 형태소 분석 및 불용어 제거
    tokens = mecab.morphs(text)  # 형태소 분석
    filtered_tokens = [word for word in tokens if word not in stop_words]  # 불용어 제거

    # 3. 정제된 단어들로 다시 결합
    preprocessed_text = " ".join(filtered_tokens)
    return preprocessed_text

# 데이터 전처리 및 학습 데이터 준비
class QADataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        
        # 질문과 답변 전처리
        question = preprocess_text(item['question'])
        answer = preprocess_text(item['answer'])

        # 질문과 답변을 토큰화
        inputs = self.tokenizer(
            question,
            answer,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # 필요한 경우, 각 입력 데이터를 'squeeze()'로 차원 축소
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }

# 데이터셋 준비
dataset = QADataset(data)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# 데이터셋에서 샘플 확인
for batch in dataloader:
    print(batch)
    break


ImportError: Unable to convert output to PyTorch tensors format, PyTorch is not installed.

In [25]:
ls "/mnt/c/fintech_service/test/023.국회 회의록 기반 지식검색 데이터/3.개방데이터/1.데이터/Training/02.라벨링데이터/"


ls: '/mnt/c/fintech_service/test/023.국회 회의록 기반 지식검색 데이터/3.개방데이터/1.데이터/Training/02.라벨링데이터/'에 접근할 수 없음: 그런 파일이나 디렉터리가 없습니다


In [13]:
# ! pip install transformers
!pip install torch

  Using cached torch-2.4.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.

In [14]:
from konlpy.tag import Mecab

# Mecab 형태소 분석기 로드
mecab = Mecab()

# 샘플 문장 형태소 분석
print(mecab.morphs("학교에서 사용하는 교육 프로그램은 무엇인가요?"))

['학교', '에서', '사용', '하', '는', '교육', '프로그램', '은', '무엇', '인가요', '?']


In [19]:
from konlpy.tag import Mecab

# Mecab 형태소 분석기 로드 (Mecab 설치 경로를 명시적으로 지정)
mecab = Mecab(dicpath=r"C:\mecab-ko-windows-x64\mecab\share\mecab-ko-dic")

# 샘플 문장 형태소 분석
print(mecab.morphs("학교에서 사용하는 교육 프로그램은 무엇인가요?"))


Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/